### Read and Write Vector Data with OGR

In OGR, when reading a vector layer, you access elements in the following order:

 1. Source
 1. Layer
 1. Feature

In [34]:
from osgeo import ogr, osr

In [25]:
# Datasource
ds = ogr.Open('data/census/tl_2019_06_tract.shp', 1)
# Layer
layer = ds.GetLayer()
extent = layer.GetExtent() # xmin, xmax, ymin, ymax

In [26]:
extent

(-124.48200299999999, -114.131211, 32.528832, 42.009502999999995)

In [27]:
feature = layer.GetFeature(1)

In [28]:
layer_def = layer.GetLayerDefn()

In [29]:
fd = layer_def.GetFieldDefn(1)

In [30]:
fd.GetName()

'COUNTYFP'

In [17]:
for i in range(layer_def.GetFieldCount()):
    field_def = layer_def.GetFieldDefn(i)
    print(field_def.GetName())

STATEFP
COUNTYFP
TRACTCE
GEOID
NAME
NAMELSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON


In [20]:
for feat in layer:
    print(feature.GetField('STATEFP'))

In [31]:
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource('output/out_test.shp')
out_layer = ds.CopyLayer(layer, 'out_test')

# Add a field to the new layer
new_field = ogr.FieldDefn('PERIMETER', ogr.OFTReal)
new_field.SetWidth(10)
new_field.SetPrecision(3)
out_layer.CreateField(new_field)

for feat in out_layer:
    geom = feat.GetGeometryRef()
    perimeter = geom.Boundary().Length()
    feat.SetField('PERIMETER', perimeter)
    out_layer.SetFeature(feature)

out_layer = out_ds = None

In [32]:
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(extent[0], extent[2])
ring.AddPoint(extent[1], extent[2])
ring.AddPoint(extent[1], extent[3])
ring.AddPoint(extent[0], extent[3])
ring.AddPoint(extent[0], extent[2])

In [33]:
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

0

In [37]:
# create a spatial reference 
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

# or extract it from a layer
srs = layer.GetSpatialFilter()

In [39]:
out_ds = driver.CreateDataSource('output/extent.shp')
out_layer = out_ds.CreateLayer('extent', srs)

feature = ogr.Feature(out_layer.GetLayerDefn())
feature.SetGeometry(poly)
out_layer.CreateFeature(feature)

out_ds = out_layer = feature = None